In [1]:
import pandas as pd
import numpy as np

#minimizing RMSE
import optuna
import numpy
from numpy import array
import math

#LSTM scores
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

#LSTM
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

#Neuron Parameters
from kerastuner.tuners import RandomSearch

C:\Users\dell\AppData\Local\Temp/ipykernel_79336/1794188618.py:24: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
x=pd.read_csv('n50.csv',parse_dates=['Date'],index_col='Date')
x = x.loc["2016-01-01" :]                         #Since 2016-01-01, 5y(1234rows till 2020-12-31), + year 2021's rows (till 30th of April)
y=x.copy()                                        #deep copy
x.reset_index(drop=True, inplace=True)
x

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,HDFC,...,JSWSTEEL,KOTAKBANK,NESTLEIND,RELIANCE,SHREECEM,TCS,TATASTEEL,TECHM,TITAN,WIPRO
0,850.608887,602.348328,1978.543457,1374.428223,1108.503052,385.106506,528.506653,82.550529,792.340698,1169.114624,...,93.927940,725.564270,5311.719238,486.556549,11121.029297,1082.013306,219.908173,461.389832,340.842285,202.975677
1,852.593201,598.059875,1954.307739,1368.077148,1096.297607,385.152100,519.647644,78.515373,794.607056,1130.328369,...,95.546753,703.665161,5205.810547,476.948639,10804.255859,1061.057739,219.481003,458.151581,340.212952,203.431641
2,871.807617,592.664673,1940.843384,1365.361816,1094.199951,383.717926,515.715759,80.314178,784.385376,1123.546631,...,99.058762,706.109497,5199.016113,481.668732,10867.756836,1051.810791,234.346603,463.031219,336.630768,203.139801
3,858.013977,603.474854,1933.413330,1381.469727,1084.378296,383.080475,517.997192,78.320908,779.852783,1123.453613,...,96.781441,701.969116,5251.443359,494.631134,11188.242188,1066.431152,229.604980,465.870178,336.776001,202.483200
4,844.413879,603.380981,1921.943726,1356.088257,1074.604370,375.636505,512.706055,74.528839,758.761841,1095.722656,...,93.868484,690.046875,5177.484863,485.550232,10725.758789,1061.796509,213.500595,453.183197,333.096954,200.568176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,2557.899902,4736.549805,10001.750000,3526.107910,3784.350098,918.745850,1404.800049,348.350006,2360.649902,2509.800049,...,655.799988,1759.650024,16779.318359,1937.849976,28062.599609,3085.706299,940.750000,962.200012,1479.849976,480.299988
1310,2574.350098,4865.049805,10091.349609,3528.100342,3908.949951,918.795288,1438.699951,366.250000,2379.850098,2518.399902,...,665.900024,1750.300049,16688.214844,1988.650024,28098.550781,3116.754150,977.750000,969.250000,1495.099976,485.049988
1311,2614.550049,5280.899902,10489.299805,3465.881348,3882.600098,913.799988,1476.800049,362.600006,2406.550049,2577.000000,...,662.650024,1811.449951,16543.800781,1997.300049,28687.550781,3108.892822,971.400024,977.400024,1508.849976,489.299988
1312,2613.449951,5484.850098,11176.549805,3456.067871,3910.850098,909.549988,1472.500000,372.149994,2407.600098,2538.850098,...,726.500000,1805.000000,16502.550781,2024.050049,28444.349609,3100.085693,1031.349976,976.900024,1506.800049,489.850006


In [3]:
def create_dataset(dataset, time_step=1):         # convert an array of values into a dataset matrix which will be used to train the lstm model.
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step):
		a = dataset[i:(i+time_step), 0]               #i=0, 0,1,2,3-----(timesteps-1)  -> timesteps
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [4]:
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(x).reshape(-1,1))   
after2020=len(y.loc["2021-01-01" : ])                    
before_2021_data_length=int(len(df1)-after2020)                 #length of data before 2021
training_size=int(before_2021_data_length*0.80)                 #80% of training size, refered from Yadav et al (2020) (Science Direct)
train_data=df1[0:training_size,:]
test_data=df1[training_size:before_2021_data_length,:1]
inpdata=df1[before_2021_data_length-60:len(df1),:1]


X_train, y_train = create_dataset(train_data, 60)
x_inp, y_inp = create_dataset(inpdata, 60)
x_test, y_test = create_dataset(test_data,60)
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
x_inp = x_inp.reshape(x_inp.shape[0],x_inp.shape[1] , 1)        #reshape input to be [samples, time steps, features] which is required for LSTM
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1] , 1)

In [9]:
def build_model(hp):
  # initialising stacked lstm
  model=Sequential()
  for i in range(hp.Int('num_layers', 2, 7)):
    model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=8),
                               activation='tanh'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_squared_error',
        metrics=['mean_squared_error'])
  return model

In [10]:
tuner=RandomSearch(
    build_model,
    objective='mean_squared_error',
    max_trials=5,
    executions_per_trial=3,
    directory='LayerNeurons',
    project_name='parameters')


In [11]:
tuner.search_space_summary()


Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 7, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 8, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 8, 'sampling': None}


In [12]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [13]:
tuner.search(X_train, y_train,
             epochs=10,
             validation_data=(x_test, y_test))


Trial 5 Complete [00h 05m 42s]
mean_squared_error: 0.014974540409942469

Best mean_squared_error So Far: 0.014974540409942469
Total elapsed time: 00h 23m 36s
INFO:tensorflow:Oracle triggered exit


In [14]:
tuner.results_summary()



Results summary
Results in LayerNeurons\parameters
Showing 10 best trials
Objective(name='mean_squared_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 5
units_0: 264
learning_rate: 0.0001
units_1: 80
units_2: 416
units_3: 160
units_4: 472
units_5: 440
Score: 0.014974540409942469
Trial summary
Hyperparameters:
num_layers: 4
units_0: 336
learning_rate: 0.001
units_1: 128
units_2: 32
units_3: 32
Score: 0.014976891068120798
Trial summary
Hyperparameters:
num_layers: 5
units_0: 160
learning_rate: 0.0001
units_1: 216
units_2: 40
units_3: 312
units_4: 32
Score: 0.015012149388591448
Trial summary
Hyperparameters:
num_layers: 6
units_0: 96
learning_rate: 0.0001
units_1: 360
units_2: 368
units_3: 448
units_4: 80
units_5: 32
Score: 0.015016817798217138
Trial summary
Hyperparameters:
num_layers: 3
units_0: 232
learning_rate: 0.01
units_1: 448
units_2: 200
units_3: 368
units_4: 288
Score: 0.015100117462376753


In [16]:
#Hyperparameters:
#num_layers: 5
#units_0: 264
#units_1: 80
#units_2: 416
#units_3: 160
#units_4: 472
#units_5: 440
#learning_rate: 0.0001
#Score: 0.014974540409942469